#Code and Data imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import seaborn as sns
import ast
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import h5py
import scipy
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import xgboost
import math
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import  tree, linear_model
from sklearn.metrics import explained_variance_score, r2_score, mean_squared_error, median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data_mix = pd.read_csv("drive/My Drive/data_mix_without_residential.csv")

In [ ]:
print(data_mix.shape)
data_mix.head()

(2921, 17)


,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kWh),TotalGHGEmissions,GHGEmissionsIntensity
0,NonResidential,Hotel,7,DOWNTOWN,1927,1.0,12.0,0,88434,Hotel,65.0,78.9,6981428.0,2023032.0,1080307.0,249.43,2.64
1,NonResidential,Hotel,7,DOWNTOWN,1996,1.0,11.0,15064,88502,"Hotel, Parking, Restaurant",51.0,94.4,8354235.0,0.0,1144563.0,263.51,2.38
2,NonResidential,Hotel,7,DOWNTOWN,1969,1.0,41.0,0,961990,"Hotel, Parking, Swimming Pool",18.0,96.6,73130656.0,19660404.0,14583930.0,2061.48,1.92
3,NonResidential,Hotel,7,DOWNTOWN,1926,1.0,10.0,0,61320,Hotel,1.0,460.4,28229320.0,23458518.0,811521.0,1936.34,31.38
4,NonResidential,Hotel,7,DOWNTOWN,1980,1.0,18.0,12460,107430,"Hotel, Parking, Swimming Pool",67.0,120.1,14829099.0,0.0,1777841.0,507.70,4.02


#Change categorical variables to dummy variables


In [ ]:
L = []
for value in data_mix['YearBuilt']:
  L.append(2016-value)
Number_of_Years = pd.Series(L)
print(Number_of_Years)

0       89
1       20
2       47
3       90
4       36
        ..
2916    61
2917    87
2918     3
2919     1
2920     1
Length: 2921, dtype: int64


In [ ]:
buildingTypeDummy = pd.get_dummies(data_mix["BuildingType"])
PrimaryPropertyTypeDummy = pd.get_dummies(data_mix["PrimaryPropertyType"])
NeighborhoodDummy = pd.get_dummies(data_mix["Neighborhood"])
CouncilDistrictCodeDummy = pd.get_dummies(data_mix["CouncilDistrictCode"])

In [ ]:
le = preprocessing.LabelEncoder()
buildingLabel = le.fit_transform(data_mix["BuildingType"])
primaryPropertyTypeLabel = le.fit_transform(data_mix["PrimaryPropertyType"])
neighborhoodLabel = le.fit_transform(data_mix["Neighborhood"])

In [ ]:
categorical_data = pd.concat([buildingTypeDummy, PrimaryPropertyTypeDummy, NeighborhoodDummy, CouncilDistrictCodeDummy], axis=1, ignore_index=True)

In [ ]:
buildingLabel = pd.Series(buildingLabel)
primaryPropertyTypeLabel = pd.Series(primaryPropertyTypeLabel)
neighborhoodLabel = pd.Series(neighborhoodLabel)

In [ ]:
categorical_data_label = pd.concat([buildingLabel, primaryPropertyTypeLabel, neighborhoodLabel], axis=1, ignore_index=True)

In [ ]:
data_mix2 = pd.concat([data_mix, categorical_data_label], axis=1, ignore_index=True)

In [ ]:
data_mix2.head() 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NonResidential,Hotel,7,DOWNTOWN,1927,1.0,12.0,0,88434,Hotel,65.0,78.9,6981428.0,2023032.0,1080307.0,249.43,2.64,0,4,5
1,NonResidential,Hotel,7,DOWNTOWN,1996,1.0,11.0,15064,88502,"Hotel, Parking, Restaurant",51.0,94.4,8354235.0,0.0,1144563.0,263.51,2.38,0,4,5
2,NonResidential,Hotel,7,DOWNTOWN,1969,1.0,41.0,0,961990,"Hotel, Parking, Swimming Pool",18.0,96.6,73130656.0,19660404.0,14583930.0,2061.48,1.92,0,4,5
3,NonResidential,Hotel,7,DOWNTOWN,1926,1.0,10.0,0,61320,Hotel,1.0,460.4,28229320.0,23458518.0,811521.0,1936.34,31.38,0,4,5
4,NonResidential,Hotel,7,DOWNTOWN,1980,1.0,18.0,12460,107430,"Hotel, Parking, Swimming Pool",67.0,120.1,14829099.0,0.0,1777841.0,507.70,4.02,0,4,5


In [ ]:
data_mix2.columns = list(data_mix.columns) + ["BuildingTypeLabel", "PrimaryPropertyTypeLabel", "NeighborhoodLabel"]

In [ ]:
data_mix2 = data_mix2.drop(["BuildingType", "PrimaryPropertyType", "Neighborhood","YearBuilt"], axis=1)
data_mixCols = list(data_mix2.columns) + ["Number Of Years"]

In [ ]:
data_mix2 = pd.concat([data_mix2, Number_of_Years], axis=1, ignore_index=True)

In [ ]:
data_mix2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,7,1.0,12.0,0,88434,Hotel,65.0,78.9,6981428.0,2023032.0,1080307.0,249.43,2.64,0,4,5,89
1,7,1.0,11.0,15064,88502,"Hotel, Parking, Restaurant",51.0,94.4,8354235.0,0.0,1144563.0,263.51,2.38,0,4,5,20
2,7,1.0,41.0,0,961990,"Hotel, Parking, Swimming Pool",18.0,96.6,73130656.0,19660404.0,14583930.0,2061.48,1.92,0,4,5,47
3,7,1.0,10.0,0,61320,Hotel,1.0,460.4,28229320.0,23458518.0,811521.0,1936.34,31.38,0,4,5,90
4,7,1.0,18.0,12460,107430,"Hotel, Parking, Swimming Pool",67.0,120.1,14829099.0,0.0,1777841.0,507.70,4.02,0,4,5,36


In [ ]:
data_mix2.columns = data_mixCols

In [ ]:
data_mix2.head()

,CouncilDistrictCode,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kWh),TotalGHGEmissions,GHGEmissionsIntensity,BuildingTypeLabel,PrimaryPropertyTypeLabel,NeighborhoodLabel,Number Of Years
0,7,1.0,12.0,0,88434,Hotel,65.0,78.9,6981428.0,2023032.0,1080307.0,249.43,2.64,0,4,5,89
1,7,1.0,11.0,15064,88502,"Hotel, Parking, Restaurant",51.0,94.4,8354235.0,0.0,1144563.0,263.51,2.38,0,4,5,20
2,7,1.0,41.0,0,961990,"Hotel, Parking, Swimming Pool",18.0,96.6,73130656.0,19660404.0,14583930.0,2061.48,1.92,0,4,5,47
3,7,1.0,10.0,0,61320,Hotel,1.0,460.4,28229320.0,23458518.0,811521.0,1936.34,31.38,0,4,5,90
4,7,1.0,18.0,12460,107430,"Hotel, Parking, Swimming Pool",67.0,120.1,14829099.0,0.0,1777841.0,507.70,4.02,0,4,5,36


#Prédiction des émissions de CO2

##Linear Regression

###Dummy Regression

In [ ]:
_d = ["SiteEnergyUse(kBtu)"]
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix2["TotalGHGEmissions"]
X = data_mix2.drop([var for var in list(data_mix2.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
Y = Y[~np.isnan(Y)]
X = X[~np.isnan(X).any(axis=1)]
#Remove NaN values
Y = Y.reshape(Y.shape[0], 1)
Y = Y[:min(X.shape[0], Y.shape[0]),:]
X = X[:min(X.shape[0], Y.shape[0]), :]
Y.shape, X.shape

((2918, 1), (2918, 1))

In [ ]:
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X, Y)
DummyRegressor()
print(dummy_regr.predict(X))
print(dummy_regr.score(X, Y))

[164.76364291 164.76364291 164.76364291 ... 164.76364291 164.76364291
 164.76364291]
0.0


###Cas 2: Multivariate Linear Regression

####Preparing data for model

In [ ]:
dict_errors = {}

In [ ]:
_d = ['Number Of Years', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)']
#_d = _d + list(categorical_data_l.columns)
data_mix = data_mix2
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix["TotalGHGEmissions"]
X = data_mix.drop([var for var in list(data_mix.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
#get where y is nan
nanY = [i for i, val in enumerate(np.isnan(Y)) if val == True]
#get where x is nan
nanX = [i for i, val in enumerate(np.isnan(X).any(axis=1)) if val == True]
toRemove = set(nanX + nanY)
Y = np.delete(Y, list(toRemove))
X = np.delete(X, list(toRemove), 0)
Y = Y.reshape(Y.shape[0], 1)

In [ ]:
X.shape, Y.shape

((2921, 4), (2921,))

In [ ]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
print(scaler_x.fit(X))
Xscale=scaler_x.transform(X)
print(scaler_y.fit(Y))
Yscale=scaler_y.transform(Y)

StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xscale, Yscale)

###Linear Regression using Scikit Learn

In [ ]:
def calcScores(model, X_test, y_test):
  predictions = model.predict(X_test)
  return {"r2_score": r2_score(y_test, predictions) , "evs": explained_variance_score(y_test, predictions), "mse": mean_squared_error(y_test, predictions), "mae": median_absolute_error(y_test, predictions)}

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
clf = LinearRegression()
scores = cross_val_score(clf, X_test, y_test, cv=6)
scores

array([-0.84774942,  0.19280592, -0.7016548 ,  0.19448785,  0.25024827,
        0.0225983 ])

In [ ]:
from sklearn.linear_model import LinearRegression
LinearRegression = LinearRegression()
parameters = {'normalize': [True, False], 'normalize': [True, False]}
LinearRegressionGCV = GridSearchCV(LinearRegression, parameters, cv=5)
LinearRegressionGCV.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'normalize': [True, False]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [ ]:
LinearRegressionGCV.best_estimator_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [ ]:
LRScores_LinearRegressionGCV = calcScores(LinearRegressionGCV, X_test, y_test)
print(LRScores_LinearRegressionGCV)

{'r2_score': 0.17121672185121561, 'evs': 0.17122838423616682, 'mse': 0.8531453823057482, 'mae': 0.09571266560353858}


In [ ]:
#linear regression with default parameters
reg = LinearRegression().fit(X_train, y_train)
print(calcScores(reg, X_test, y_test))

{'r2_score': 0.17121672185121561, 'evs': 0.17122838423616682, 'mse': 0.8531453823057482, 'mae': 0.09571266560353847}


###Cas 3: Regression Ridge (L2 Regularization)

####Ridge using gridsearch over 'alpha'


In [ ]:
n_alphas = 200
ridge = Ridge()
parameters = {'alpha': list(np.logspace(-5, 5, 200)), 'normalize': [True, False], 'fit_intercept': [True, False]}
ridgeGCV = GridSearchCV(ridge, parameters)
ridgeGCV.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-05, 1.1226677735108135e-05,
                                   1.2603829296797274e-05,
                                   1.414991297434576e-05, 1.588565129428053e-05,
                                   1.7834308769319092e-05,
                                   2.002200371815...
                                   9.011018251665018e-05, 0.0001011637979766207,
                                   0.00011357333583431052,
                                   0.00012750512407130128,
                                   0.00014314589375234786,
                                   0.00016070528182616383,
                                   0.00018041864093920718,
              

In [ ]:
ridgeGCV.best_estimator_

Ridge(alpha=0.6669919663030128, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [ ]:
ridgeGCV.best_params_

{'alpha': 0.6669919663030128, 'fit_intercept': True, 'normalize': True}

In [ ]:
LRScores_ridge = calcScores(ridgeGCV, X_test, y_test)
LRScores_ridge

{'evs': 0.18169819163611423,
 'mae': 0.12595540686198692,
 'mse': 0.8424512624470264,
 'r2_score': 0.1816054643765208}

###Cas 4: Regression Lasso (L1 Regularization)

In [ ]:
n_alphas = 200
lasso = Lasso()
parameters = {'alpha': list(np.logspace(-5, 5, n_alphas))}
lassoGCV = GridSearchCV(lasso, parameters)
lassoGCV.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-05, 1.1226677735108135e-05,
                                   1.2603829296797274e-05,
                                   1.414991297434576e-05, 1.58...
                                   7.149428986597576e-05, 8.026433522257174e-05,
                                   9.011018251665018e-05, 0.0001011637979766207,
                                   0.00011357333583431052,
                                   0.00012750512407130128,
                                   0.00014314589375234786,
                                   0.00016070528182616383,
                                

In [ ]:
lassoGCV.best_params_

{'alpha': 0.18679135990207846}

In [ ]:
lasso = Lasso(alpha=lassoGCV.best_params_['alpha'])
lasso.fit(X_train, y_train)

Lasso(alpha=0.18679135990207846, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
LRScores_lasso = calcScores(lassoGCV, X_test, y_test)
LRScores_lasso

{'evs': 0.18688554011571712,
 'mae': 0.12082289050430323,
 'mse': 0.8371059072290009,
 'r2_score': 0.18679817960694456}

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
liste_depth = list(range(10))
RandomForestRegressor = RandomForestRegressor()
parameters = {'max_depth': liste_depth}
RandomForestRegressorGCV = GridSearchCV(RandomForestRegressor, parameters, n_jobs = 10)
RandomForestRegressorGCV.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

In [ ]:
RandomForestRegressorGCV.best_params_

{'max_depth': 3}

In [ ]:
LRScores_RandomForestRegressor = calcScores(RandomForestRegressorGCV, X_test, y_test)
LRScores_RandomForestRegressor

{'evs': 0.09838445668516593,
 'mae': 0.11105813785293506,
 'mse': 0.9293361910573736,
 'r2_score': 0.09720158966903236}

##Random forest sans GCV

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor(n_estimators = 300, max_depth=5, random_state=0)
clf = regr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [ ]:
RandomForestErros = calcScores(clf, X_test, y_test)

In [ ]:
RandomForestErros

{'evs': 0.1577037944294406,
 'mae': 0.07852970845353327,
 'mse': 0.8685584761292775,
 'r2_score': 0.15624375863719042}

##XGBOOST

In [ ]:
xgb = xgboost.XGBRegressor() 
parameters = {'learning_rate': [0.001, 0.01, 0.1, 1], 'gamma': list(np.logspace(-5, 5, 50)), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
xgbGSCV = GridSearchCV(xgb, parameters, verbose=1, n_jobs=10)
xgbGSCV.fit(X_train, y_train)

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   11.6s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   26.9s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   52.0s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  1.5min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed:  3.1min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed:  4.2min
[Parallel(n_jobs=10)]: Done 3180 tasks      | elapsed:  5.5min
[Parallel(n_jobs=10)]: Done 4030 tasks      | elapsed:  6.9min
[Parallel(n_jobs=10)]: Done 4980 tasks      | elapsed:  8.5min
[Parallel(n_jobs=10)]: Done 6030 tasks      | elapsed: 10.2min
[Parallel(n_jobs=10)]: Done 7180 tasks      | elapsed: 12.1min
[Parallel(n_jobs=10)]: Done 8430 tasks      | elapsed: 14.2min


[20:42:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=10)]: Done 9000 out of 9000 | elapsed: 15.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, r...
                                   0.07543120063354623, 0.1206792640639329,
                                   0.19306977288832497, 0.30888435964774846,
                                   0.49417133613238384, 0.7906043210907702,
                                   1.2648552168552958, 2.0235896477251556,
                     

In [ ]:
xgbGSCV.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1206792640639329,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
print(calcScores(xgbGSCV, X_train, y_train))
print(calcScores(xgbGSCV, X_test, y_test))

{'r2_score': 0.9055465602298314, 'evs': 0.9055466569735795, 'mse': 0.08668360156709087, 'mae': 0.07674073292432587}
{'r2_score': 0.6334198971516991, 'evs': 0.6338048777461021, 'mse': 0.45677945292705463, 'mae': 0.07508224503181621}


In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
cv_results = cross_validate(xgbGSCV, Xscale, Yscale, cv=5)
sorted(cv_results.keys())
cv_results['test_score']

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.1, subsample=0.75, colsample_bytree=1, max_depth=5)

In [ ]:
xgb.fit(X_train, y_train)
print(calcScores(xgb, X_train, y_train))
print(calcScores(xgb, X_test, y_test))

[16:27:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'r2_score': 0.9375280747377622, 'evs': 0.9375293522425692, 'mse': 0.06184987897217107, 'mae': 0.07180692453863693}
{'r2_score': 0.3541704615349708, 'evs': 0.35524340826806755, 'mse': 0.6648137130961495, 'mae': 0.07564910639401784}


#Prédictions de la consommation totale d'énergie

###Data preparation

In [ ]:
_d = ["SiteEnergyUse(kBtu)"]
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix2["SiteEnergyUse(kBtu)"]
X = data_mix2.drop([var for var in list(data_mix2.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
Y = Y[~np.isnan(Y)]
X = X[~np.isnan(X).any(axis=1)]
#Remove NaN values
Y = Y.reshape(Y.shape[0], 1)
Y = Y[:min(X.shape[0], Y.shape[0]),:]
X = X[:min(X.shape[0], Y.shape[0]), :]
Y.shape, X.shape

In [ ]:
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X, Y)
DummyRegressor()
print(dummy_regr.predict(X))
print(dummy_regr.score(X, Y))

####Preparing data for model

In [ ]:
dict_errors = {}

In [ ]:
_d = ['Number Of Years', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)']
#_d = _d + list(categorical_data_l.columns)
data_mix = data_mix2
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix["TotalGHGEmissions"]
X = data_mix.drop([var for var in list(data_mix.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
#get where y is nan
nanY = [i for i, val in enumerate(np.isnan(Y)) if val == True]
#get where x is nan
nanX = [i for i, val in enumerate(np.isnan(X).any(axis=1)) if val == True]
toRemove = set(nanX + nanY)
Y = np.delete(Y, list(toRemove))
X = np.delete(X, list(toRemove), 0)
Y = Y.reshape(Y.shape[0], 1)

In [ ]:
_d = ['Number Of Years','NumberofBuildings','NumberofFloors','PropertyGFABuilding(s)']
#_d = _d + list(categorical_data.columns)
data_mix = data_mix2
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix["SiteEnergyUse(kBtu)"]
X = data_mix.drop([var for var in list(data_mix.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
#get where y is nan
nanY = [i for i, val in enumerate(np.isnan(Y)) if val == True]
#get where x is nan
nanX = [i for i, val in enumerate(np.isnan(X).any(axis=1)) if val == True]
toRemove = set(nanX + nanY)
Y = np.delete(Y, list(toRemove))
X = np.delete(X, list(toRemove), 0)
Y = Y.reshape(Y.shape[0], 1)
Y.shape, X.shape

((2914, 1), (2914, 4))

In [ ]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
print(scaler_x.fit(X))
Xscale=scaler_x.transform(X)
print(scaler_y.fit(Y))
Yscale=scaler_y.transform(Y)

StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xscale, Yscale)

####Multivariant Linear Regression Functions

In [ ]:
def forward(X, W, b):
  #Returns Prediction based on aX + b 
  Yhat = np.dot(X, W.T) + b
  return Yhat

In [ ]:
def cost(Yhat, Y):
  #returns scalar
  diff = np.power(Yhat - Y, 2)
  return diff.sum()/Yhat.shape[0]

In [ ]:
def calculateDerivatives(X, Y, Yhat):
  n = X.shape[0]
  diff = Y - Yhat
  mult = np.dot(X.T, diff)
  Da = (-2/n) * mult
  Db = (-2/n) * diff.sum()
  return Da, Db

In [ ]:
def updateParams(W, b, Da, Db, lr):
  W_new = W - lr * Da.T
  b_new = b - lr * Db
  return W_new, b_new

In [ ]:
def uniLinearRegression(X, Y, lr=0.001, epochs=500):
  W = np.random.rand(1,X.shape[1])
  b = 0
  errors = []
  for i in range(epochs):
    Yhat = forward(X, W, b)
    Yhat = Yhat/Yhat.mean()
    if i%100 == 0: print('Interation '+str(i), W, b)
    err = cost(Yhat, Y)
    errors.append(err)
    Da, Db = calculateDerivatives(X, Y, Yhat)
    W, b = updateParams(W, b, Da, Db, lr)
  return W, b, errors

In [ ]:
W, b, errors = uniLinearRegression(X_train, y_train, epochs=500)

In [ ]:
plt.plot(errors)
plt.ylabel('cost')
plt.xlabel('iterations (per tens)')
plt.title("MSE")
plt.show()

##Linear Regression

###Cas 2: Multivariate Linear Regression

###Linear Regression using Scikit Learn

In [ ]:
def calcScores(model, X_test, y_test):
  predictions = model.predict(X_test)
  return {"r2_score": r2_score(y_test, predictions) , "evs": explained_variance_score(y_test, predictions), "mse": mean_squared_error(y_test, predictions), "mae": median_absolute_error(y_test, predictions)}

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)
reg.coef_
reg.intercept_

array([-0.00387505])

In [ ]:
LRScores_linear_regression = calcScores(reg, X_test, y_test)
print(LRScores_linear_regression)

{'r2_score': 0.5804762393604774, 'evs': 0.5806765553979103, 'mse': 0.5024796466062168, 'mae': 0.10447913459287506}


###Cas 3: Regression Ridge (L2 Regularization)

####La baseline : une régression classique


In [ ]:
n_alphas = 200
ridge = Ridge()
parameters = {'alpha': list(np.logspace(-5, 5, n_alphas))}
ridgeGCV = GridSearchCV(ridge, parameters)
ridgeGCV.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-05, 1.1226677735108135e-05,
                                   1.2603829296797274e-05,
                                   1.414991297434576e-05, 1.588565129428053e-05,
                                   1.7834308769319092e-05,
                                   2.002200371815...
                                   7.149428986597576e-05, 8.026433522257174e-05,
                                   9.011018251665018e-05, 0.0001011637979766207,
                                   0.00011357333583431052,
                                   0.00012750512407130128,
                                   0.00014314589375234786,
                                   0.00016070528182

In [ ]:
sorted(ridgeGCV.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_alpha',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [ ]:
ridgeGCV.best_params_

{'alpha': 60.80224261649427}

In [ ]:
LRScores_ridge = calcScores(ridgeGCV, X_test, y_test)
LRScores_ridge

{'evs': 0.5725194341801576,
 'mae': 0.10819993080088056,
 'mse': 0.5122561175943035,
 'r2_score': 0.5723137955631876}

###Cas 4: Regression Lasso (L1 Regularization)

In [ ]:
n_alphas = 200
lasso = Lasso()
parameters = {'alpha': list(np.logspace(-5, 5, n_alphas))}
lassoGCV = GridSearchCV(lasso, parameters)
lassoGCV.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-05, 1.1226677735108135e-05,
                                   1.2603829296797274e-05,
                                   1.414991297434576e-05, 1.58...
                                   7.149428986597576e-05, 8.026433522257174e-05,
                                   9.011018251665018e-05, 0.0001011637979766207,
                                   0.00011357333583431052,
                                   0.00012750512407130128,
                                   0.00014314589375234786,
                                   0.00016070528182616383,
                                

In [ ]:
lassoGCV.best_params_

{'alpha': 0.02072921779595372}

In [ ]:
LRScores_lasso = calcScores(lassoGCV, X_test, y_test)
LRScores_lasso

{'evs': 0.573892668403294,
 'mae': 0.106713059055138,
 'mse': 0.5105885527034175,
 'r2_score': 0.5737060571181779}

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
liste_depth=list(range(10))
RandomForestRegressor = RandomForestRegressor()
parameters = {'max_depth': liste_depth}
RandomForestRegressorGCV = GridSearchCV(RandomForestRegressor, parameters, n_jobs=10)
RandomForestRegressorGCV.fit(X_train, y_train)

In [ ]:
RandomForestRegressorGCV.best_params_

{'max_depth': 9}

In [ ]:
LRScores_RandomForestRegressor = calcScores(RandomForestRegressorGCV, X_test, y_test)
LRScores_RandomForestRegressor

{'evs': 0.6257298517973007,
 'mae': 0.06891134883465276,
 'mse': 0.4489301056775423,
 'r2_score': 0.6251851244718443}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor()
clf = regr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [ ]:
RandomForestErros=calcScores(clf, X_test, y_test)
RandomForestErros

{'evs': 0.5936058622372433,
 'mae': 0.04584290296322957,
 'mse': 0.48735861252034873,
 'r2_score': 0.5931008961546524}

##XGBOOST

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0.001, subsample=0.75, colsample_bytree=1, max_depth=X_train.shape[1])
xgb.fit(X_train, y_train)

[17:11:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.75, verbosity=1)

In [ ]:
LRScores_XGBOOST = calcScores(clf, X_test, y_test)

In [ ]:
LRScores_XGBOOST

{'evs': 0.5936058622372433,
 'mae': 0.04584290296322957,
 'mse': 0.48735861252034873,
 'r2_score': 0.5931008961546524}

In [ ]:
xgb = xgboost.XGBRegressor(subsample=0.75, colsample_bytree=1)
parameters = {'learning_rate': [0.001, 0.01, 0.1, 1], 'gamma': list(np.logspace(-5, 5, 50)), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
xgbGSCV = GridSearchCV(xgb, parameters, verbose=1, n_jobs=10)
xgbGSCV.fit(X_train, y_train)

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.7s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   24.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   48.4s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed:  3.0min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed:  4.1min
[Parallel(n_jobs=10)]: Done 3180 tasks      | elapsed:  5.4min
[Parallel(n_jobs=10)]: Done 4030 tasks      | elapsed:  6.8min
[Parallel(n_jobs=10)]: Done 4980 tasks      | elapsed:  8.3min
[Parallel(n_jobs=10)]: Done 6030 tasks      | elapsed: 10.0min
[Parallel(n_jobs=10)]: Done 7180 tasks      | elapsed: 11.8min
[Parallel(n_jobs=10)]: Done 8430 tasks      | elapsed: 13.8min
[Parallel(n_jobs=10)]: Done 9000 out of 9000 | elapsed: 14.8min finished


[17:32:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, r...
                                   0.07543120063354623, 0.1206792640639329,
                                   0.19306977288832497, 0.30888435964774846,
                                   0.49417133613238384, 0.7906043210907702,
                                   1.2648552168552958, 2.0235896477251556,
                     

In [ ]:
predictionsTrain = xgbGSCV.predict(X_train)
predictions = xgbGSCV.predict(X_test)
print('Explained variance score: Train: ' + str(explained_variance_score(y_train, predictionsTrain)) + ' Test: ' + str(explained_variance_score(y_test, predictions)))
print('R2 score: Train: ' + str(r2_score(y_train, predictionsTrain)) + ' Test: ' + str(r2_score(y_test, predictions)))

Explained variance score: Train: 0.9192363207545433 Test: 0.6507204757282095
R2 score: Train: 0.9192344576443997 Test: 0.6498285069479928


#Evaluation de l'intérêt de "l'ENERGY STAR Score" pour la prédiction d'émissions

In [ ]:
data_ESS = data_mix.dropna(subset=['ENERGYSTARScore'])

In [ ]:
data_ESS.shape

###Modèle sans la variable ENERGYSTARScore

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0.001, subsample=0.75, colsample_bytree=1, max_depth=X_train.shape[1])

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
LRScores_XGBOOST_sansESS = calcScores(clf, X_test, y_test)

###Modèle avec la variable ENERGYSTARScore

In [ ]:
_d = ['CouncilDistrictCode','YearBuilt','NumberofBuildings','NumberofFloors','PropertyGFAParking','PropertyGFABuilding(s)','ENERGYSTARScore']
#_d = _d + list(categorical_data.columns)
data_mix = data_ESS
#To predict variable : TotalGHGEmissions
#Features : correlated quantitative variables
#To remove (BuildingType, PrimaryPropertyType)
Y = data_mix["SiteEnergyUse(kBtu)"]
X = data_mix.drop([var for var in list(data_mix.columns) if var not in _d], axis=1)
Y = np.array(Y)
X = np.array(X)
Y = Y[~np.isnan(Y)]
X = X[~np.isnan(X).any(axis=1)]
#Remove NaN values
Y = Y.reshape(Y.shape[0], 1)
Y = Y[:min(X.shape[0], Y.shape[0]),:]
X = X[:min(X.shape[0], Y.shape[0]), :]
Y.shape, X.shape

In [ ]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
print(scaler_x.fit(X))
Xscale=scaler_x.transform(X)
print(scaler_y.fit(Y))
Yscale=scaler_y.transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xscale, Yscale)

In [ ]:
xgbWithScore = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0.001, subsample=0.75, colsample_bytree=1, max_depth=X_train.shape[1])

In [ ]:
xgbWithScore.fit(X_train, y_train)

In [ ]:
LRScores_XGBOOST_avecESS = calcScores(xgbWithScore, X_test, y_test)

In [ ]:
print(LRScores_XGBOOST_sansESS)
print(LRScores_XGBOOST_avecESS)